# Importing libraries

In [ ]:
import os, glob, json, csv, subprocess, sys, re
from urllib.request import urlopen
from itertools import chain
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Counter
import pandas as pd
import plotly.graph_objs as go
import plotly
from plotly.offline import plot, init_notebook_mode, iplot
import itertools

%matplotlib inline

# Defining directories

In [ ]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/openjpa/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_openjpa/'

# Defining git command function

In [ ]:
def execute_command(cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Finding log messages containing bug

In [ ]:
#Applying git command to find commit log messages containing words of bugs, fix, defect or patch
words = ['bug', 'fix', 'defect', 'patch']
logs = []
for n, word in enumerate(words):
    git_cmd = "git log --all --grep='" + word + "' --oneline"
    log = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('b"','').replace('(','',1).replace("\\'","'").split("\\n"))[:-1]

    logs.append(log)

com_logs = [item for sublist in logs for item in sublist]
com_logs

In [ ]:
#separating the commit id and the log messages
commit = []
for xx in range(0,len(com_logs)):
    tmp = []
    comm = com_logs[xx].split()
    word = ' '.join(comm[1:])
    tmp.extend([comm[0],word])
    commit.append(tmp)

for item in commit:
    print (item)

# Capturing bug ids

In [ ]:
#Capturing bug ids which are identified as bug links in the log messages
commitbugs = []
buglinks = []

for xx in range(0,len(com_logs)):
    tmp = []
    comm = com_logs[xx].split()
    splitres = (str(comm[1:]))
    letter = re.findall(r"OPENJPA+[-][0-9]+", splitres)
    if letter != []:
        commitbugs.extend(letter)
    else:
        pass

commitbugs = set(commitbugs)
for p,q in enumerate(commitbugs):
    buglinks.append(q)
buglinks = sorted(buglinks, reverse=True)
print (buglinks)
print ("\nNumber of bug id: " + str(len(buglinks)))

In [ ]:
#Synchronizing the bug link with the bug report from JIRA ITS database
errorlinks = []
id_type = []
for a,b in enumerate(buglinks):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " %(a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        types = soup.find('span', attrs={'id':'type-val'}).text.replace("\n",'').replace(" ",'').split(",")
        types = sorted(types)
        types.insert(0, b)
        id_type.append(types)
    except:
        errorlinks.append(b)

print("\nExtraction has been completed.")

In [ ]:
#Finding out if there were any error from the sync process above
type_of_id = id_type
errorlinks

In [ ]:
#Resync for the error bug ids identified
error_links = []
for a,b in enumerate(errorlinks):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " %(a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        types = soup.find('span', attrs={'id':'type-val'}).text.replace("\n",'').replace(" ",'').split(",")
        types = sorted(types)
        types.insert(0, b)
        type_of_id.append(types)
    except:
        error_links.append(b)

In [ ]:
type_of_id.sort()
error_links

In [ ]:
bug_links = []
for d, types in enumerate (type_of_id):
    if types[1] == 'Bug':
        bug_links.append(types[0])

print ("Number of bug id: " + str(len(bug_links)))
print (bug_links)

In [ ]:
with open (analyze_dir + "01_code_document/txt_files/candidate_bug_links.txt", mode="wt", encoding="utf-8") as myfile:
    myfile.write('\n'.join(bug_links))
print ("File candidate_bug_links.txt has been created")

# Extracting logs related to bug links

In [ ]:
com_log = []
for no, bug_link in enumerate(bug_links):
    sys.stdout.write("\r%i " %(no+1) + "Extracting bug id: " + bug_link)
    sys.stdout.flush()
    git_cmd = "git log --all --oneline | grep -w '" + bug_link + "'"
    temp = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('b"','').replace('(','',1)
            .replace("\\\\n","\n").split("\\n"))[:-1]

    com_log.append(temp)

print ("\n")
for item in com_log:
    print(item)

Capturing bug id, and commit id related to the bug id

In [ ]:
commits = []
for x in range(0,len(com_log)):
    com = []
    com.append(bug_links[x])
    for z in range(0,len(com_log[x])):
        temp = str(com_log[x][z].split()[0:1]).replace("['",'').replace("']",'')
        com.append(temp)
    commits.append(com)
print (commits)

# Capturing bug-introducing change commit id

In [ ]:
bugintro_cid = []
for jj, kk in enumerate(commits):
    if (len(kk) > 1):
        bi_cid = kk[len(kk)-1:]
        bi_cid.insert(0,kk[0])
        bugintro_cid.append(bi_cid)
print (len(bugintro_cid), "\n")
print (bugintro_cid)

# Capturing candidate bug fix commit id

In [ ]:
bugfix_cid = []
for jj, kk in enumerate(commits):
    if (len(kk) > 2):
        bugfix_cid.append(kk[0:len(kk)-1])
print (bugfix_cid)

Counting the number of candidate bug fix commit id

In [ ]:
summ = 0
for b, bfc in enumerate(bugfix_cid):
    summ = summ + (len(bfc)-1)
print ("The number of candidate bug fix commit id = %i" %summ)

In [ ]:
bugintro = go.Bar(
    x = ['Bug-introducing change commits'],
    y = [len(bugintro_cid)],
    name = 'Bug-introducing change commits',
    text = [len(bugintro_cid)],
    textposition = 'auto'
)

bugfix = go.Bar(
    x = ['Candidate bug-fix commits'],
    y = [summ],
    name = 'Candidate bug-fix commits',
    text = [summ],
    textposition = 'auto'
)

data = [bugintro, bugfix]
layout = go.Layout(
    title = 'The number of commits in OPENJPA Project'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

# Extracting parent_id of bug-intro commits

In [ ]:
parent_id = []
for v, w in enumerate(bugintro_cid):
    sys.stdout.write("\rExtracting parent id from bug-intro commit: {} / {}".format((v+1), len(bugintro_cid)))
    sys.stdout.flush()
    if w != []:
        git_cmd = "git log --pretty=%P -n1 " + w[1]
        temp = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('(','',1).split("\\n"))[:-1]
        if ' ' in temp[0]:
            pr = temp[0].split(' ')
            pr = pr[len(pr)-1:]
            temps = [w,pr]
        else:
            temps = [w,temp]
        newlist = list(chain.from_iterable(temps))

    parent_id.append(newlist)

In [ ]:
parent_id

Saving parent id in CSV file

In [ ]:
with open(analyze_dir + "02_extracting_commitid/parentid_of_bugintro.csv", 'w') as parent:
    header = ['bug_id','bugintro_commitID','parent_id']
    writers = csv.writer(parent, delimiter=',')
    writers.writerow(header)
    for item in parent_id:
        writers.writerow(item)

dfparent = pd.read_csv(analyze_dir + "02_extracting_commitid/parentid_of_bugintro.csv")
dfparent = dfparent[header]
dfparent

# Extracting commit_id of candidate bug fix commits

In [ ]:
bugfix_cid

In [ ]:
commit_idfix = []
for y in range(0,len(bugfix_cid)):
    sys.stdout.write("\rExtracting commit id from candidate bug-fix commit: {} / {}".format((y+1), len(bugfix_cid)))
    sys.stdout.flush()
    if w != []:
        for s in range(1,len(bugfix_cid[y])):
            temps = [bugfix_cid[y][0],bugfix_cid[y][s]]         
            commit_idfix.append(temps)
commit_idfix

Saving into CSV file

In [ ]:
with open(analyze_dir + "02_extracting_commitid/commitid_of_candidatebugfix.csv", 'w') as parentfix:
    cols = ['bug_id','candidate_bugfix_commitID']
    writers = csv.writer(parentfix, delimiter=',')
    writers.writerow(cols)
    for item in commit_idfix:
        writers.writerow(item)

dfcommitfix = pd.read_csv(analyze_dir + "02_extracting_commitid/commitid_of_candidatebugfix.csv")
dfcommitfix = dfcommitfix[cols]
dfcommitfix